To complete this task, follow these steps:

Create a new Jupyter Notebook file and name it Vacation_Search.ipynb.
Import the WeatherPy_vacation.csv file from Part 1 as a new DataFrame.
Filter the DataFrame for minimum and maximum temperature preferences, and if the rain or snow accumulation is 0 inches or not using conditional statements. Do the following:
Prompt the customer for the minimum temperature preference.
Prompt the customer for the maximum temperature preference.
Prompt the customer to answer if he or she would like it to be raining or not, using input("Do you want it to be raining? (yes/no) ").
Prompt the customer to answer if he or she would like it to be snowing or not, using input("Do you want it to be snowing? (yes/no) ").
Add the cities to a marker layer map with a pop-up marker for each city that includes:
Hotel name
City
Country
Current weather description with the maximum temperature
Save and upload the new DataFrame as WeatherPy_vacation.csv.
Save and upload the new marker layer map as WeatherPy_vacation_map.png.

In [1]:
#Import Dependencies.
import pandas as pd
import gmaps
import requests
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("Data/WeatherPy_challenge.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain(inches),Snow(inches)
0,Atuona,PF,-9.80,-139.03,80.64,70,7,11.25,clear sky,0,0
1,Tasiilaq,GL,65.61,-37.64,37.40,86,90,14.99,light intensity shower rain,0,0
2,Zhigansk,RU,66.77,123.37,36.97,73,100,20.54,overcast clouds,0,0
3,Ushuaia,AR,-54.80,-68.30,37.40,79,40,5.26,scattered clouds,0,0
4,Punta Arenas,CL,-53.15,-70.92,37.00,74,100,8.05,overcast clouds,0,0


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
#Ask for customer weather preferrences.
#max_temp
max_temp=float(input("What is the maximum acceptable temperature for your trip? "))
#min_temp
min_temp=float(input("what is the minimum acceptable temperature for your trip? "))
#rain
rain_preferrence=input("Do you want it to be raining? (yes/no) ")
#snow
snow_preferrence=input("Do you want it to be snowing? (yes/no) ")

What is the maximum acceptable temperature for your trip? 92
what is the minimum acceptable temperature for your trip? 50
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [5]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                       

#want rain and not snow.
if rain_preferrence=='yes'and snow_preferrence=='no':
    preferred_cities_df = preferred_cities_df.loc[(city_data_df['Rain(inches)']>0) & (city_data_df['Snow(inches)']==0)]
#want snow and not rain.    
elif rain_preferrence=='no'and snow_preferrence=='yes':
     preferred_cities_df = preferred_cities_df.loc[(city_data_df['Rain(inches)']==0) & (city_data_df['Snow(inches)']>0)]
#want no snow and no rain.        
elif rain_preferrence=='no'and snow_preferrence=='no':
     preferred_cities_df = preferred_cities_df.loc[(city_data_df['Rain(inches)']==0) & (city_data_df['Snow(inches)']==0)]
#want snow or rain(or instead of and as both conditions cant be true at the same time.)        
else :
     preferred_cities_df = preferred_cities_df.loc[(city_data_df['Rain(inches)']>0) or (city_data_df['Snow(inches)']>0)]
        
    
preferred_cities_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain(inches),Snow(inches)
0,Atuona,PF,-9.80,-139.03,80.64,70,7,11.25,clear sky,0,0
5,Aras,ES,42.56,-2.36,84.20,21,0,10.29,clear sky,0,0
7,Saint George,US,37.10,-113.58,84.00,23,1,4.70,clear sky,0,0
8,Vaini,TO,-21.20,-175.20,69.80,94,20,3.36,few clouds,0,0
9,Chuy,UY,-33.70,-53.46,63.82,62,0,5.77,clear sky,0,0


In [6]:
#Check for null value.
preferred_cities_df.count()

City                   447
Country                446
Lat                    447
Lng                    447
Max Temp               447
Humidity               447
Cloudiness             447
Wind Speed             447
Current Description    447
Rain(inches)           447
Snow(inches)           447
dtype: int64

In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,80.64,clear sky,-9.80,-139.03,
5,Aras,ES,84.20,clear sky,42.56,-2.36,
7,Saint George,US,84.00,clear sky,37.10,-113.58,
8,Vaini,TO,69.80,few clouds,-21.20,-175.20,
9,Chuy,UY,63.82,clear sky,-33.70,-53.46,
10,Souillac,MU,77.00,shower rain,-20.52,57.52,
11,East London,ZA,77.00,clear sky,-33.02,27.91,
13,Jawhar,SO,81.55,overcast clouds,2.78,45.50,
14,Rikitea,PF,76.37,light rain,-23.12,-134.97,
17,Busselton,AU,62.01,light rain,-33.65,115.33,


In [8]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [9]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
     hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,80.64,clear sky,-9.80,-139.03,Villa Enata
5,Aras,ES,84.20,clear sky,42.56,-2.36,Jardines de Viana Spa
7,Saint George,US,84.00,clear sky,37.10,-113.58,Best Western Plus Abbey Inn
8,Vaini,TO,69.80,few clouds,-21.20,-175.20,Keleti Beach Resort
9,Chuy,UY,63.82,clear sky,-33.70,-53.46,Nuevo Hotel Plaza
10,Souillac,MU,77.00,shower rain,-20.52,57.52,Bils' Villa
11,East London,ZA,77.00,clear sky,-33.02,27.91,Tu Casa
13,Jawhar,SO,81.55,overcast clouds,2.78,45.50,hantiwadaag village part one
14,Rikitea,PF,76.37,light rain,-23.12,-134.97,Pension Maro'i
17,Busselton,AU,62.01,light rain,-33.65,115.33,Observatory Guest House


In [11]:
# Create the output file.csv.
output_data_file= "Data/WeatherPy_vacation.csv"

#Export the city data into a csv.
hotel_df.to_csv(output_data_file, index=False)

In [12]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [15]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]

#Set the variable for weight.
max_temp = hotel_df["Max Temp"]

#Set the varaible for plot.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.5)

#Set the variables for heatmap.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

#Set the variable for marker.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

#Add the heat layer.
fig.add_layer(heat_layer)

#Add the marker layer.
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig



Figure(layout=FigureLayout(height='420px'))